In [1]:
import utils_clean
import utils
import numpy as np
import pandas as pd

In [2]:
from pathlib import Path
import json
test = {i:2 for i in range(13)}

try:
    with open(Path("final_experiment/"+"test.json"), 'w') as f:
        json.dump(test, f, indent=1)
except:
    print("COULDNT SAVE SYNTH COMPAS")
print(test)

test["twenty"] = 20

try:
    with open(Path("final_experiment/"+"test2.json"), 'w') as f:
        json.dump(test, f, indent=1)
except:
    print("COULDNT SAVE SYNTH COMPAS")

{0: 2, 1: 2, 2: 2, 3: 2, 4: 2, 5: 2, 6: 2, 7: 2, 8: 2, 9: 2, 10: 2, 11: 2, 12: 2}


In [ ]:
def run(response, missing_col, sensitive, dataset = "recid", n_runs = 10, robust = True, with_mcar = True):
    full_results = {}
    for run in tqdm(range(n_runs)):
        results = {"mar": {metr: {i: [] for i in IMPUTATIONS} for metr in METRICS},
                   "mcar": {metr: {i: [] for i in IMPUTATIONS} for metr in METRICS}}
        np.random.seed(run*13)
        if dataset =="simple":
            data = utils.load_synthetic("simple")
            alpha = SIMPLE_ALPHA
        elif dataset =="adult":
            data = utils.load_adult()
            raise NotImplementedError("FOR THE FUTURE")
        elif dataset == "recid_synth":
            data = utils.load_synthetic()
            alpha = RECID_ALPHA
        else:
            data = utils.load_compas_alt()
            alpha = RECID_ALPHA
        
        train = data["train"]
        test = data["test"]
        class_0_test = test[test[sensitive] == 0]
        class_1_test = test[test[sensitive] == 1]
        
        noise = np.random.normal(0,0.1,size = len(train))
        
        for alph, missing_pct in zip(alpha, [0, 5,10,25,50,75]):
            #print(missing_pct)
            if with_mcar:
                data_mcar = utils_clean.data_remover_cat(
                        train, missing_col, alph, noise, missing_pct = missing_pct, missingness="mcar", robust = robust)
            data_mar = utils_clean.data_remover_cat(
                    train, missing_col, alph, noise, missingness="mar", robust = robust)
            #print(data_mcar_missing.columns)
            #TODO replace MODELS[m] with xgboost and include imputations and without imputations
            for imp in IMPUTATIONS:  # , "reg"]:
                
                if imp == "coldel" and missing_col == sensitive:
                    continue
                #print("MODEL",m,"\n", "IMP", imp)
                #MCAR
                # TPR, FPR, TNR, FNR data
                #Investigate XGBoost at the fit stage and if data needs to be transformed
                if with_mcar:
                    predictions_0 = MODELS[m].fit(data_mcar.drop(response, axis=1), data_mcar[response]).predict(
                        class_0_test.drop([response, missing_col], axis=1) if imp == "coldel" else class_0_test.drop(response, axis=1))
                    predictions_1 = MODELS[m].fit(data_mcar.drop(response, axis=1), data_mcar[response]).predict(
                        class_1_test.drop([response, missing_col], axis=1) if imp == "coldel" else class_0_test.drop(response, axis=1))
                    
                    
                    cf_0 = utils.confusion_matrix(class_0_test[response], predictions_0)
                    cf_1 = utils.confusion_matrix(class_1_test[response], predictions_1)
                    results["mcar"]["tpr0"][m][imp].append(cf_0["Predicted true"][0])
                    results["mcar"]["tpr1"][m][imp].append(cf_1["Predicted true"][0])
                    results["mcar"]["tnr0"][m][imp].append(cf_0["Predicted false"][1])
                    results["mcar"]["tnr1"][m][imp].append(cf_1["Predicted false"][1])
                    
                    results[m+"_mcar_"+imp+"_" +
                            str(missing_pct)+"_0"] = cf_0
                    results[m+"_mcar_"+imp+"_" +
                            str(missing_pct)+"_1"] = cf_1

                    # Fairness metrics
                    y_hat = MODELS[m].fit(data_mcar.drop(response, axis=1), data_mcar[response]).predict(
                        test.drop([response, missing_col], axis=1) if imp == "coldel" else test.drop(response, axis=1))
                    results["mcar"]["spd"][m][imp].append(
                        utils.spd(y_hat, test[sensitive]))
                    
                    results["mcar"]["acc"][m][imp].append(
                        utils.accuracy(test[response], y_hat))
                    eo = utils.equalised_odds(y_hat, test[sensitive], test[response])
                    results["mcar"]["eosum"][m][imp].append(eo["Y=1"]+eo["Y=0"])
                # TPR, FPR, TNR, FNR data

                #MAR
                try:
                    predictions_0 = MODELS[m].fit(data_mar.drop(response, axis=1), data_mar[response]).predict(
                        class_0_test.drop([response, missing_col], axis=1) if imp == "coldel" else class_0_test.drop(response, axis=1))
                except Exception as e:
                    print("Exception: ", e)
                    print("params: ", str(missing_pct)+imp)
                    print("head: ", data_mar.head())
                    print("sum: ", data_mar.sum()-len(data_mar))
                predictions_1 = MODELS[m].fit(data_mar.drop(response, axis=1), data_mar[response]).predict(
                    class_1_test.drop([response, missing_col], axis=1) if imp == "coldel" else class_1_test.drop(response, axis=1))
                cf_0 = utils.confusion_matrix(class_0_test[response], predictions_0)
                cf_1 = utils.confusion_matrix(class_1_test[response], predictions_1)
                results["mar"]["tpr0"][m][imp].append(cf_0["Predicted true"][0])
                results["mar"]["tpr1"][m][imp].append(cf_1["Predicted true"][0])
                results["mar"]["tnr0"][m][imp].append(cf_0["Predicted false"][1])
                results["mar"]["tnr1"][m][imp].append(cf_1["Predicted false"][1])
                    
                results[m+"_mar_"+imp+"_" +
                        str(missing_pct)+"_0"] = cf_0
                results[m+"_mar_"+imp+"_" +
                        str(missing_pct)+"_1"] = cf_1

                # Fariness metrics
                y_hat = MODELS[m].fit(data_mar.drop(response, axis=1), data_mar[response]).predict(
                    test.drop([response, missing_col], axis=1) if imp == "coldel" else test.drop(response, axis=1))
                results["mar"]["spd"][m][imp].append(
                    utils.spd(y_hat, test[sensitive]))
                results["mar"]["acc"][m][imp].append(
                    utils.accuracy(test[response], y_hat))

                eo = utils.equalised_odds(y_hat, test[sensitive], test[response])
                results["mar"]["eosum"][m][imp].append(eo["Y=1"]+eo["Y=0"])
                #TODO add to thesis that missingness was not applied to the test data
                #print(imp)
        
        full_results[str(run)] = results
    temp_delta = collections.defaultdict(list)
    avg = collections.defaultdict(list)
    std = {}
    
    #Collecting all observations across runs
    for miss in ["mcar", "mar"]:
        if not with_mcar and miss =="mcar":
            continue
        for key in [str(n) for n in range(n_runs)]:
            for imp in IMPUTATIONS:
                for metric in METRICS:
                    temp_delta[miss+"|"+metric+"|"+"xgboost"+"|"+imp] += full_results[key][miss][metric]["xgboost"][imp]
                    avg[miss+"|"+metric+"|"+"xgboost"+"|"+imp].append(full_results[key][miss][metric]["xgboost"][imp])
    
    #Averaging
    for key, value in avg.items():
        avg[key] = [float(n) for n in np.mean(value, axis = 0)]  
        std[key] =  [float(n) for n in np.std(value, axis = 0)] 
          
          
    try:
        utils_clean.save("./data/", "testresults.pickle", results)
    except Exception as e:
        print("Couldn't save data with exception: ", e)  
    try:
        utils_clean.save("./data/", "testaverages.pickle", avg)
    except Exception as e:
        print("Couldn't save data with exception: ", e)  
    return {"Full data": full_results, "Averaged results": avg, "Standard deviation": std}